# Data cleaning and exploratory analysis

## First look at data set

In [1]:
# Import packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Regex for string matching
from re import match

In [2]:
# Import data sets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

all_data = [train, test]

Let's get a look at the data:

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Summary of the data;

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


Check that the test data contains the same information:

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


Contains all the same information, except survival (as we expect).

## Quick cleaning

The PassengerId doesn't give any information so I'll drop it

In [6]:
train = train.drop(columns = 'PassengerId')
test = test.drop(columns = 'PassengerId')

Move 'Survived' column in train to the far right:

In [7]:
# Move 'Survived' to far right in train
train['survived'] = train['Survived']
train = train.drop(columns = 'Survived')

Make column names snake_case:

In [8]:
# Make column names snake_case
col_names = ['pclass', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare', 'cabin', 'embarked']

train.columns = col_names + ['survived']
test.columns = col_names

## Inspecting variables

### pclass

In [9]:
train.pclass.value_counts()

3    491
1    216
2    184
Name: pclass, dtype: int64

### sex

In [10]:
train.sex.value_counts()

male      577
female    314
Name: sex, dtype: int64

### age

In [11]:
train.age.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: age, dtype: float64

Ages are fractional if < 1. Note that 177 ages are missing.

### sibsp and parch

In [12]:
train[['sibsp', 'parch']].describe()

,sibsp,parch
count,891.000000,891.000000
mean,0.523008,0.381594
std,1.102743,0.806057
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,0.000000
max,8.000000,6.000000


In [13]:
train.sibsp.value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: sibsp, dtype: int64

In [14]:
train.parch.value_counts()

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: parch, dtype: int64

Q: Would it be possible to use names to group families?

### ticket

In [15]:
train.ticket.nunique()

681

All passengers have tickets, but only 681 of these are unique.

In [16]:
ticket_value_counts = train.ticket.value_counts()
len(ticket_value_counts[ticket_value_counts == 1])

547

547 tickets have only one passenger associated, so 134 have multiple passengers. The most is 7 passengers with the same ticket value.

Do the tickets contain any information I can use?

In [17]:
train.ticket.head(30)

0            A/5 21171
1             PC 17599
2     STON/O2. 3101282
3               113803
4               373450
5               330877
6                17463
7               349909
8               347742
9               237736
10             PP 9549
11              113783
12           A/5. 2151
13              347082
14              350406
15              248706
16              382652
17              244373
18              345763
19                2649
20              239865
21              248698
22              330923
23              113788
24              349909
25              347077
26                2631
27               19950
28              330959
29              349216
Name: ticket, dtype: object

Most are a 6 digit number. Some start with some letters. Most numbers start with 1, 2, or 3  == pclass?

Begin by isolating those which are numbers of at least 6 digits (this is revisionism - I found that those with 7 digits behave the same as those with 6).

In [18]:
six_digit_tickets = train[train.ticket.str.contains('^\d{6}\d*$', regex = True)]

In [19]:
six_digit_tickets.ticket.str[0].value_counts()

3    272
2     87
1     72
Name: ticket, dtype: int64

With these, the first digit is either 1, 2, or 3 - suggests pclass. Check if these match.

In [20]:
(pd.to_numeric(six_digit_tickets.ticket.str[0]) == six_digit_tickets.pclass).value_counts()

True    431
dtype: int64

All true - where the ticket is a number of 6 or more digits, the first number equals pclass. Unclear if I'll be able to get anything from the rest. Note that some of the 6 digit tickets are repeated, some not, as with all tickets.

Examine tickets which are 7 or more digits:

Let's take a look at the remaining tickets. First isolate those that are numbers but less than 6 digits.

In [21]:
less_than_six_digit_ticket = train[train.ticket.str.contains('^\d{,5}$', regex = True)]

In [22]:
less_than_six_digit_ticket.head(10)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,survived
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,0
19,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C,1
26,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C,0
27,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,C23 C25 C27,S,0
36,3,"Mamee, Mr. Hanna",male,NaN,0,0,2677,7.2292,NaN,C,1
39,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,2651,11.2417,NaN,C,1
40,3,"Ahlin, Mrs. Johan (Johanna Persdotter Larsson)",female,40.0,1,0,7546,9.4750,NaN,S,0
41,2,"Turpin, Mrs. William John Robert (Dorothy Ann ...",female,27.0,1,0,11668,21.0000,NaN,S,0
47,3,"O'Driscoll, Miss. Bridget",female,NaN,0,0,14311,7.7500,NaN,Q,1
48,3,"Samaan, Mr. Youssef",male,NaN,2,0,2662,21.6792,NaN,C,0


With these, it is not the case that the first digit equals pclass.

Now let's look at those tickets which contain something other than a number:

In [23]:
non_number_tickets = train[train.ticket.str.contains('\D', regex = True)]

In [24]:
non_number_tickets.ticket.head(20)

0            A/5 21171
1             PC 17599
2     STON/O2. 3101282
10             PP 9549
12           A/5. 2151
30            PC 17601
31            PC 17569
33          C.A. 24579
34            PC 17604
37          A./5. 2152
43       SC/Paris 2123
45     S.C./A.4. 23567
51          A/4. 39886
52            PC 17572
56          C.A. 31026
58          C.A. 34651
59             CA 2144
64            PC 17605
66          C.A. 29395
67           S.P. 3464
Name: ticket, dtype: object

Strip away the numbers at the end:

In [33]:
ticket_letters = non_number_tickets.ticket.str.split()
ticket_letters = ticket_letters.str[0]
ticket_letters = ticket_letters.str.replace('.', '')

In [31]:
ticket_letters.value_counts()

PC          60
CA          41
A/5         19
SOTON/OQ    15
STON/O      12
W/C         10
SC/PARIS     7
A/4          6
STON/O2      6
SOC          5
C            5
FCC          5
LINE         4
SC/Paris     4
PP           3
SC/AH        3
SO/PP        3
P/PP         2
A5           2
SOTON/O2     2
SW/PP        2
WE/P         2
SC           1
CA/SOTON     1
SOP          1
SC/A4        1
A4           1
SCO/W        1
A/S          1
SO/C         1
Fa           1
FC           1
WEP          1
SP           1
Name: ticket, dtype: int64

'SOTON' and variants probably refer to Southampton. Unclear what the rest means or what we could do with it.